In [1]:
import pandas as pd
from ingest import *
from itertools import compress

# Get Team Stats

In [ ]:
# To pull from the url - NOT NEEDED IF CVS ALREADY IMPORTED LOCALLY, JUST PULL CSV IN BELOW CELL
#p_shoot = getPreviousSeasons(getShootingStatsPlayers, 5)
#p_pass = getPreviousSeasons(getPassingStatsPlayers, 5)
#p_passType = getPreviousSeasons(getPassTypesStatsPlayers, 5)
#p_gsc = getPreviousSeasons(getGoalShotCreationStatsPlayers, 5)
#p_def = getPreviousSeasons(getDefenseStatsPlayers, 5)
#p_poss = getPreviousSeasons(getPossessionStatsPlayers, 5)
#p_time = getPreviousSeasons(getPlayingTimeStatsPlayers, 5)
#p_misc = getPreviousSeasons(getMiscStatsPlayers, 5)

In [2]:
t_shoot = pd.read_csv('raw/shooting-squads.csv')
t_pass = pd.read_csv('raw/passing-squads.csv')
t_passType = pd.read_csv('raw/pass-types-squads.csv')
t_gsc = pd.read_csv('raw/goal-shot-creating-squads.csv')
t_def = pd.read_csv('raw/defense-squads.csv')
t_poss = pd.read_csv('raw/possession-squads.csv')
t_time = pd.read_csv('raw/playing-time-squads.csv')
t_misc = pd.read_csv('raw/miscellaneous-squads.csv')

In [3]:
#demo
t_shoot

,Rk,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,...,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,League,Season,TeamID
0,1,Alavés,24,11.0,6,109,29,26.6,9.91,2.64,...,2,2,10.0,8.4,0.08,-4.0,-4.4,La Liga,2021-2022,37745278530914180
1,2,Angers,22,12.0,17,119,37,31.1,9.92,3.08,...,3,3,14.7,12.4,0.11,2.3,1.6,Ligue 1,2021-2022,66587352409382849
2,3,Arminia,22,10.0,6,119,37,31.1,11.90,3.70,...,1,1,8.9,8.2,0.07,-2.9,-3.2,Bundesliga,2021-2022,60726784699523350
3,4,Arsenal,26,10.0,12,141,44,31.2,14.10,4.40,...,0,1,12.5,11.6,0.09,-0.5,0.4,Premier League,2021-2022,5838645288327307
4,5,Aston Villa,23,10.0,13,113,34,30.1,11.30,3.40,...,2,2,11.0,9.6,0.09,2.0,1.4,Premier League,2021-2022,14348480492050895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,94,Watford,29,38.0,42,437,128,29.3,11.50,3.37,...,3,6,47.4,42.6,0.10,-5.4,-3.6,Premier League,2017-2018,73743918793554398
486,95,Werder Bremen,25,34.0,34,437,143,32.7,12.85,4.21,...,0,0,39.9,39.9,0.09,-5.9,-5.9,Bundesliga,2017-2018,59303898890876434
487,96,West Brom,24,38.0,31,383,106,27.7,10.08,2.79,...,1,1,34.4,33.7,0.09,-3.4,-3.7,Premier League,2017-2018,26775906631723373
488,97,West Ham,27,38.0,48,369,128,34.7,9.71,3.37,...,2,4,39.9,36.8,0.10,8.1,9.2,Premier League,2017-2018,57680956417459581


# Check Dimensions
Want to see if each table has same number of rows (players)

In [4]:
dfs = [t_shoot,t_pass,t_passType,t_gsc,t_def,t_poss,t_time,t_misc]
for i in dfs:
    print(i.shape)

(490, 24)
(490, 29)
(490, 32)
(490, 23)
(490, 30)
(490, 32)
(490, 27)
(490, 23)


They do

# Some initial cleaning
Get rid of "Rk" column - it's unnecessary and will mess up merging the tables since it's basically just an index

In [5]:
dfs = [t_shoot,t_pass,t_passType,t_gsc,t_def,t_poss,t_time,t_misc]
for i in dfs:
    i.drop(i.columns[0], axis=1, inplace=True)

In [11]:
t_misc.head()

,Squad,# Pl,90s,CrdY,CrdR,2CrdY,Fls,Fld,Off,Crs,...,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League,Season,TeamID
0,Alavés,24,11.0,23.0,1.0,1,148,133,20,135,...,2,3,0,806,275,185,59.8,La Liga,2021-2022,37745278530914180
1,Angers,22,12.0,18.0,0.0,0,142,146,14,112,...,3,3,0,847,122,163,42.8,Ligue 1,2021-2022,66587352409382849
2,Arminia,22,10.0,16.0,1.0,1,137,93,10,105,...,1,3,0,890,230,264,46.6,Bundesliga,2021-2022,60726784699523350
3,Arsenal,26,10.0,14.0,1.0,0,94,81,18,89,...,1,0,0,768,161,180,47.2,Premier League,2021-2022,5838645288327307
4,Aston Villa,23,10.0,23.0,1.0,0,106,134,14,144,...,1,2,1,812,175,196,47.2,Premier League,2021-2022,14348480492050895


Just for viewing purposes, I want all the String/categorical variables at the beginning.

Moving "League", "Season", and "PlayerID" columns below.

In [7]:
first_column = t_shoot.pop('League')
t_shoot.insert(0, 'League', first_column)
first_column = t_shoot.pop('Season')
t_shoot.insert(0, 'Season', first_column)
first_column = t_shoot.pop('TeamID')
t_shoot.insert(0, 'TeamID', first_column)
t_shoot.head()

,TeamID,Season,League,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,...,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,37745278530914180,2021-2022,La Liga,Alavés,24,11.0,6,109,29,26.6,...,0.14,16.1,3,2,2,10.0,8.4,0.08,-4.0,-4.4
1,66587352409382849,2021-2022,Ligue 1,Angers,22,12.0,17,119,37,31.1,...,0.38,17.5,6,3,3,14.7,12.4,0.11,2.3,1.6
2,60726784699523350,2021-2022,Bundesliga,Arminia,22,10.0,6,119,37,31.1,...,0.14,19.1,3,1,1,8.9,8.2,0.07,-2.9,-3.2
3,5838645288327307,2021-2022,Premier League,Arsenal,26,10.0,12,141,44,31.2,...,0.27,17.4,6,0,1,12.5,11.6,0.09,-0.5,0.4
4,14348480492050895,2021-2022,Premier League,Aston Villa,23,10.0,13,113,34,30.1,...,0.32,17.2,4,2,2,11.0,9.6,0.09,2.0,1.4


In [10]:
#test 
t_shoot.loc[t_shoot['TeamID']==5838645288327307]

,TeamID,Season,League,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,...,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
3,5838645288327307,2021-2022,Premier League,Arsenal,26,10.0,12,141,44,31.2,...,0.27,17.4,6,0,1,12.5,11.6,0.09,-0.5,0.4
101,5838645288327307,2020-2021,Premier League,Arsenal,29,38.0,53,455,141,31.0,...,0.33,16.9,23,6,6,53.5,49.0,0.11,-0.5,-2.0
199,5838645288327307,2019-2020,Premier League,Arsenal,29,38.0,56,401,144,35.9,...,0.37,15.9,19,3,3,49.2,46.9,0.12,6.8,6.1
297,5838645288327307,2018-2019,Premier League,Arsenal,28,38.0,69,466,159,34.1,...,0.41,16.2,11,4,5,58.0,54.1,0.12,11.0,10.9
395,5838645288327307,2017-2018,Premier League,Arsenal,30,38.0,73,590,227,38.5,...,0.30,17.4,30,4,5,68.4,64.6,0.11,4.6,4.4


# Merging the tables

I used all the string/categorical variables for 2 reasons: 
- more exact joins (just PlayerID wouldn't know which to merge when a player moved teams within a season)
- don't have to deal with cleaning duplicate columns

outer and inner join shouldn't matter for this (they both should, and do, come out to the same result here since all the individual tables have the same number of rows).

In [9]:
# put the various tables into a list to iterate merging
dfs = [t_shoot,t_gsc,t_pass,t_passType,t_def,t_poss,t_misc,t_time]

In [16]:
for j in range(len(dfs)):
    if j == 0:
        combined = dfs[j]
    else:
        combined = pd.merge(combined, dfs[j], on=['TeamID', 'Season', 'League', 'Squad', '# Pl', '90s'], how='outer')

In [17]:
combined

,TeamID,Season,League,Squad,# Pl,90s,Gls,Sh_x,SoT,SoT%,...,unSub,PPM,onG,onGA,+/-,+/-90,onxG,onxGA,xG+/-,xG+/-90
0,37745278530914180,2021-2022,La Liga,Alavés,24,11.0,6,109,29,26.6,...,75,0.91,6,14,-8,-0.73,10.0,14.8,-4.8,-0.44
1,66587352409382849,2021-2022,Ligue 1,Angers,22,12.0,17,119,37,31.1,...,57,1.42,18,15,3,0.25,14.7,13.6,1.1,0.09
2,60726784699523350,2021-2022,Bundesliga,Arminia,22,10.0,6,119,37,31.1,...,43,0.50,6,16,-10,-1.00,8.9,19.1,-10.1,-1.01
3,5838645288327307,2021-2022,Premier League,Arsenal,26,10.0,12,141,44,31.2,...,60,1.70,12,13,-1,-0.10,12.5,15.9,-3.4,-0.34
4,14348480492050895,2021-2022,Premier League,Aston Villa,23,10.0,13,113,34,30.1,...,60,1.00,14,19,-5,-0.50,11.0,15.9,-4.9,-0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,73743918793554398,2017-2018,Premier League,Watford,29,38.0,42,437,128,29.3,...,155,1.08,44,64,-20,-0.53,47.4,57.9,-10.6,-0.28
486,59303898890876434,2017-2018,Bundesliga,Werder Bremen,25,34.0,34,437,143,32.7,...,140,1.24,37,40,-3,-0.09,39.9,48.5,-8.7,-0.26
487,26775906631723373,2017-2018,Premier League,West Brom,24,38.0,31,383,106,27.7,...,157,0.82,31,56,-25,-0.66,34.4,48.5,-14.1,-0.37
488,57680956417459581,2017-2018,Premier League,West Ham,27,38.0,48,369,128,34.7,...,167,1.11,48,68,-20,-0.53,39.9,59.2,-19.3,-0.51


In [18]:
combined.to_csv('Combined_Team.csv')

The next 2 things I want to do:
- Add a new column for a player that checks which league he was in last year. 
    - Since there are cases where players are transferred/loaned out midseason and can have 2 rows for the same sesaon, I think for ease of analysis, we may be best off just to use the team that he played more minutes for as the single row for that player for that season.
- Convert all stats to per 90 so I can start exploratory data analysis

# Filters:
- remove the ongoing 2021-2022 season 
- (DOES NOT APPLY FOR TEAMS) remove player seasons under a specified threshold of minutes/90's played (currently set to 720 mins / 8.0 90s, but can change that later on
- (DOES NOT APPLY FOR TEAMS) when a player has two rows for the same season (due to transfer/loan), only use the row/team where they played more minutes
- (DOES NOT APPLY FOR TEAMS) remove players with only one season left in the dataset after all above previous filters have been applied

In [21]:
cprac = combined.copy()
cprac['90s'] = cprac['90s'].astype(float)
cprac = cprac.loc[(cprac.Season != "2021-2022")] #& (cprac['90s'] > 8.0)]
cprac = cprac.sort_values(['Squad', 'Season'],ascending=[1,1])
#cprac = cprac.drop_duplicates(subset = ['PlayerID','Season'], keep='first').reset_index(drop = True)
#for i in range(1,len(cprac)-1):
#    if (i-1) in cprac.index:
#        if cprac.loc[i]['PlayerID'] != cprac.loc[i-1]['PlayerID'] and cprac.loc[i]['PlayerID'] != cprac.loc[i+1]['PlayerID']:
#            cprac.drop(labels=i, axis=0, inplace=True)
#    else:
#        if cprac.loc[i]['PlayerID'] != cprac.loc[i+1]['PlayerID']:
#            cprac.drop(labels=i, axis=0, inplace=True)
cprac = cprac.reset_index(drop = True)
#if cprac.iloc[-1]['PlayerID'] != cprac.iloc[-2]['PlayerID']:
#    cprac.drop(labels=(len(cprac)-1), axis=0, inplace=True)
cprac

,TeamID,Season,League,Squad,# Pl,90s,Gls,Sh_x,SoT,SoT%,...,unSub,PPM,onG,onGA,+/-,+/-90,onxG,onxGA,xG+/-,xG+/-90
0,37745278530914180,2017-2018,La Liga,Alavés,30,38.0,40,393,137,34.9,...,154,1.24,40,50,-10,-0.26,38.8,53.2,-14.4,-0.38
1,37745278530914180,2018-2019,La Liga,Alavés,26,38.0,39,419,118,28.2,...,153,1.32,39,50,-11,-0.29,37.3,51.0,-13.7,-0.36
2,37745278530914180,2019-2020,La Liga,Alavés,32,38.0,34,299,99,33.1,...,187,1.03,34,59,-25,-0.66,37.0,53.2,-16.2,-0.43
3,37745278530914180,2020-2021,La Liga,Alavés,30,38.0,35,337,99,29.4,...,254,1.00,36,57,-21,-0.55,42.4,50.8,-8.3,-0.22
4,20069478421376018,2017-2018,Ligue 1,Amiens,30,38.0,36,372,110,29.6,...,155,1.18,37,42,-5,-0.13,35.3,55.4,-20.2,-0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,85159484018358879,2019-2020,Bundesliga,Wolfsburg,26,34.0,48,466,155,33.3,...,199,1.44,48,46,2,0.06,47.3,40.1,7.3,0.21
388,85159484018358879,2020-2021,Bundesliga,Wolfsburg,26,34.0,57,475,162,34.1,...,173,1.79,61,37,24,0.71,49.5,42.0,7.4,0.22
389,24001928852251903,2018-2019,Premier League,Wolves,21,38.0,46,467,141,30.2,...,156,1.50,47,46,1,0.03,49.0,38.1,10.8,0.29
390,24001928852251903,2019-2020,Premier League,Wolves,21,38.0,49,453,142,31.3,...,187,1.55,51,40,11,0.29,47.1,34.8,12.3,0.32


One additional filter:
- (DOES NOT APPLY FOR TEAMS since we don't care about YoY trend) Remove rows where teams have non-consequtive seasons 
- (DOES NOT APPLY FOR TEAMS) Finally, add in a column for the league the player was in during the previous season. Impute 0 for all missing values.

In [22]:
#df_w_prev_season = pd.DataFrame()
#for i in list(set(cprac['Squad'])):
#    holder = cprac.loc[cprac['Squad']==i].sort_values(['Season'],ascending=[1])
#    holder = holder.reset_index(drop = True)
#    if int(holder.loc[0]['Season'][-2:]) != (int(holder.loc[1]['Season'][-2:])-1):
#        holder.drop(labels=0, axis=0, inplace=True)
#    for j in range(1,len(holder)-1):
#        if (j-1) in holder.index:
#            if int(holder.loc[j]['Season'][-2:]) != (int(holder.loc[j-1]['Season'][-2:])+1) and int(holder.loc[j]['Season'][-2:]) != (int(holder.loc[j+1]['Season'][-2:])-1):
#                holder.drop(labels=j, axis=0, inplace=True)
#        else:
#            if int(holder.loc[j]['Season'][-2:]) != (int(holder.loc[j+1]['Season'][-2:])-1):
#                holder.drop(labels=j, axis=0, inplace=True)
#    holder = holder.reset_index(drop = True)
#    if len(holder) > 1:
#        if int(holder.iloc[-1]['Season'][-2:]) != (int(holder.iloc[-2]['Season'][-2:])+1):
#            holder.drop(labels=(len(holder)-1), axis=0, inplace=True)
#    else:
#        holder.drop(labels=0, axis=0, inplace=True)
#    holder['prev_season_league'] = holder['League'].shift()
#    df_w_prev_season = df_w_prev_season.append(holder)
#df_w_prev_season = df_w_prev_season.sort_values(['PlayerID', 'Season'],ascending=[1,1])
#df_w_prev_season = df_w_prev_season.reset_index(drop = True)
#df_w_prev_season.fillna(0, inplace = True)

In [23]:
#df_w_prev_season

In [289]:
cprac.to_csv('Combined_Team_Cleaned.csv')

# Convert season gross stats in per 90 basis when applicable

In [24]:
cprac.columns.tolist()

['TeamID',
 'Season',
 'League',
 'Squad',
 '# Pl',
 '90s',
 'Gls',
 'Sh_x',
 'SoT',
 'SoT%',
 'Sh/90',
 'SoT/90',
 'G/Sh',
 'G/SoT',
 'Dist',
 'FK',
 'PK',
 'PKatt',
 'xG',
 'npxG',
 'npxG/Sh',
 'G-xG',
 'np:G-xG',
 'SCA',
 'SCA90',
 'PassLive',
 'PassDead',
 'Drib',
 'Sh_y',
 'Fld_x',
 'Def',
 'GCA',
 'GCA90',
 'PassLive.1',
 'PassDead.1',
 'Drib.1',
 'Sh.1',
 'Fld.1',
 'Def.1',
 'Total Cmp',
 'Total Att',
 'Total Cmp%',
 'Total TotDist',
 'Total PrgDist',
 'Short Cmp',
 'Short Att',
 'Short Cmp%',
 'Medium Cmp',
 'Medium Att',
 'Medium Cmp%',
 'Long Cmp',
 'Long Att',
 'Long Cmp%',
 'Ast',
 'xA',
 'A-xA',
 'KP',
 '1/3_x',
 'PPA',
 'CrsPA',
 'Prog_x',
 'Att_x',
 'Pass Types: Live',
 'Pass Types: Dead',
 'Pass Types: FK',
 'Pass Types: TB',
 'Pass Types: Press',
 'Pass Types: Sw',
 'Pass Types: Crs',
 'Pass Types: CK',
 'Corner Kicks: In',
 'Corner Kicks: Out',
 'Corner Kicks: Str',
 'Height: Ground',
 'Height: Low',
 'Height: High',
 'Body Parts: Left',
 'Body Parts: Right',
 'Body P

In [27]:
final_df = cprac.copy()
all_columns = cprac.columns.tolist()
not_90 = ['TeamID','Season','League','Squad','# Pl','90s','SoT%','Sh/90','SoT/90','G/Sh','G/SoT','Dist','npxG/Sh',
 'SCA90','GCA90','Total Cmp%','Short Cmp%','Medium Cmp%','Long Cmp%','Vs Dribbles: Tkl%','Pressures: %','Poss',
 'Succ%','Rec%','Won%','Age','MP','Mn/MP','Min%','Mn/Start','Mn/Sub','PPM','+/-','+/-90','xG+/-','xG+/-90']
col_tf = [elem not in not_90 for elem in all_columns]
col_to90 = list(compress(all_columns, col_tf))
#len(col_to90)
#final_df = getPer90Columns(final_df, col_to90)
for name in col_to90:
    final_df[name] = round(final_df[name].astype(float) / final_df['90s'].astype(float),2)
final_df

,TeamID,Season,League,Squad,# Pl,90s,Gls,Sh_x,SoT,SoT%,...,unSub,PPM,onG,onGA,+/-,+/-90,onxG,onxGA,xG+/-,xG+/-90
0,37745278530914180,2017-2018,La Liga,Alavés,30,38.0,1.05,10.34,3.61,34.9,...,4.05,1.24,1.05,1.32,-10,-0.26,1.02,1.40,-14.4,-0.38
1,37745278530914180,2018-2019,La Liga,Alavés,26,38.0,1.03,11.03,3.11,28.2,...,4.03,1.32,1.03,1.32,-11,-0.29,0.98,1.34,-13.7,-0.36
2,37745278530914180,2019-2020,La Liga,Alavés,32,38.0,0.89,7.87,2.61,33.1,...,4.92,1.03,0.89,1.55,-25,-0.66,0.97,1.40,-16.2,-0.43
3,37745278530914180,2020-2021,La Liga,Alavés,30,38.0,0.92,8.87,2.61,29.4,...,6.68,1.00,0.95,1.50,-21,-0.55,1.12,1.34,-8.3,-0.22
4,20069478421376018,2017-2018,Ligue 1,Amiens,30,38.0,0.95,9.79,2.89,29.6,...,4.08,1.18,0.97,1.11,-5,-0.13,0.93,1.46,-20.2,-0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,85159484018358879,2019-2020,Bundesliga,Wolfsburg,26,34.0,1.41,13.71,4.56,33.3,...,5.85,1.44,1.41,1.35,2,0.06,1.39,1.18,7.3,0.21
388,85159484018358879,2020-2021,Bundesliga,Wolfsburg,26,34.0,1.68,13.97,4.76,34.1,...,5.09,1.79,1.79,1.09,24,0.71,1.46,1.24,7.4,0.22
389,24001928852251903,2018-2019,Premier League,Wolves,21,38.0,1.21,12.29,3.71,30.2,...,4.11,1.50,1.24,1.21,1,0.03,1.29,1.00,10.8,0.29
390,24001928852251903,2019-2020,Premier League,Wolves,21,38.0,1.29,11.92,3.74,31.3,...,4.92,1.55,1.34,1.05,11,0.29,1.24,0.92,12.3,0.32


In [28]:
final_df.to_csv('Combined_Team_Per90.csv')

In [36]:
eda = final_df.copy()
eda = eda.groupby('League', as_index=False).mean().round(2)

In [37]:
eda.to_csv('League_averages.csv')